In [1]:
!pip install biopandas -q
!pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html -q
!pip install transformers --upgrade -q

In [2]:
import os
import numpy as np
import pandas as pd
from biopandas.pdb import PandasPdb
from biopandas.mmcif import PandasMmcif
import torch
import dgl
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
tokenizers.__version__: 0.14.1
transformers.__version__: 4.34.1


In [3]:
def get_distance_matrix(coords):
    #输入氨基酸残基的平均3D坐标
    #输出每两个氨基酸残基的欧式距离矩阵
    diff_tensor = np.expand_dims(coords, axis=1) - np.expand_dims(coords, axis=0)
    distance_matrix = np.sqrt(np.sum(np.power(diff_tensor, 2), axis=-1))
    return distance_matrix

def generate_graph(pdb_path, distance_threshold=8.0):
    atom_df = PandasPdb().read_pdb(pdb_path)
    atom_df = atom_df.df['ATOM']
    residue_df = atom_df.groupby('residue_number', as_index=False)[['x_coord', 'y_coord', 'z_coord', 'b_factor']].mean().sort_values('residue_number')
    coords = residue_df[['x_coord', 'y_coord', 'z_coord']].values
    distance_matrix = get_distance_matrix(coords)
    adj = distance_matrix < distance_threshold
    u, v = np.nonzero(adj)
    u, v = torch.from_numpy(u), torch.from_numpy(v)
    graph = dgl.graph((u, v), num_nodes=len(coords))
    return graph

In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

In [5]:
import torch
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

Downloading: "https://github.com/facebookresearch/esm/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [6]:
def seq_encode(seq):
    # Prepare data (single sequence)
    data = [("protein", seq)]
    
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    
    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33]
    #print(token_representations.shape)
    #print(token_representations)
    # Generate per-sequence representation
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representation = token_representations.squeeze()[1:-1, :]
    #print(sequence_representation.shape)
    return sequence_representation 

#seq = "MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFARTCKCLAEHFNVVLFDLPFAGQSRQHNPQRGLITKDDEVEILLALIERFEVNHLVSASWGGISTLLALSRNPRGIRSSVVMAFAPGLNQAMLDYVGRAQALIELDDKSAIGHLLNETVGKYLPQRLKASNHQHMASLATGEYEQARFHIDQVLALNDRGYLACLERIQSHVHFINGSWDEYTTAEDARQFRDYLPHCSFSRVEGTGHFLDLESKLAAVRVHRALLEHLLKQPEPQRAERAAGFHEMAIGYA"
#seq_encode(seq)
#print(len(seq))

In [7]:
aa_map = {'VAL': 'V', 'PRO': 'P', 'ASN': 'N', 'GLU': 'E', 'ASP': 'D', 'ALA': 'A', 'THR': 'T', 'SER': 'S',
          'LEU': 'L', 'LYS': 'K', 'GLY': 'G', 'GLN': 'Q', 'ILE': 'I', 'PHE': 'F', 'CYS': 'C', 'TRP': 'W',
          'ARG': 'R', 'TYR': 'Y', 'HIS': 'H', 'MET': 'M'}

In [8]:
aa_props = pd.read_csv('/kaggle/input/aminoacids-csv/aminoacids.csv').set_index('Letter')
aa_props2 = aa_props[['Molecular Weight', 'Residue Weight', 'pKa1', 'pKb2', 'pl4', 
         'H', 'VSC', 'P1', 'P2', 'SASA', 'NCISC', 'carbon', 'hydrogen', 'nitrogen',
       'oxygen', 'sulfur']]
propp = ['pKa1', 'pKb2', 'pl4', 'H', 'VSC', 'P1', 'P2', 'SASA', 'NCISC', 'carbon', 'hydrogen', 'nitrogen', 'oxygen', 'sulfur']
aa_props2 = aa_props2.fillna(aa_props2.mean())
min_weight = np.min(aa_props2[['Molecular Weight', 'Residue Weight']].values)
max_weight = np.max(aa_props2[['Molecular Weight', 'Residue Weight']].values)
aa_props2[['Molecular Weight', 'Residue Weight']] = (aa_props2[['Molecular Weight', 'Residue Weight']] - min_weight) / (max_weight - min_weight)
aa_props2[propp] = (aa_props2[propp]-aa_props2[propp].min()) / (aa_props2[propp].max()-aa_props2[propp].min())
aa_props2

,Molecular Weight,Residue Weight,pKa1,pKb2,pl4,H,VSC,P1,P2,SASA,NCISC,carbon,hydrogen,nitrogen,oxygen,sulfur
Letter,,,,,,,,,,,,,,,,
A,0.217761,0.095325,0.514851,0.494444,0.404255,0.805627,0.189003,0.395062,0.112469,0.168350,0.158604,0.111111,0.222222,0.000000,0.0,0.0
C,0.435589,0.313222,0.138614,0.822222,0.287860,0.721228,0.306529,0.074074,0.312958,0.325477,0.000000,0.111111,0.222222,0.000000,0.0,1.0
D,0.516782,0.394347,0.059406,0.444444,0.000000,0.416880,0.274914,1.000000,0.256724,0.396184,0.046317,0.222222,0.222222,0.000000,1.0,0.0
E,0.612040,0.489673,0.366337,0.483333,0.056320,0.457801,0.426117,0.913580,0.369193,0.550505,0.157210,0.333333,0.444444,0.000000,1.0,0.0
F,0.734747,0.612379,0.009901,0.183333,0.339174,0.951407,0.793814,0.037037,0.709046,0.755892,0.268566,0.777778,0.666667,0.000000,0.0,0.0
G,0.122435,0.000000,0.514851,0.444444,0.400501,0.769821,0.000000,0.506173,0.000000,0.000000,0.780985,0.000000,0.000000,0.000000,0.0,0.0
H,0.666599,0.544164,0.000000,0.205556,0.603254,0.544757,0.542955,0.679012,0.562347,0.641975,0.093865,0.444444,0.444444,0.666667,0.0,0.0
I,0.503669,0.381234,0.534653,0.444444,0.406758,1.000000,0.642612,0.037037,0.454768,0.521324,0.210910,0.444444,0.888889,0.000000,0.0,0.0
K,0.605653,0.483286,0.356436,0.083333,0.872340,0.263427,0.687285,0.790123,0.535452,0.772727,0.196704,0.444444,1.000000,0.333333,0.0,0.0


In [9]:
def phys_chem_encode(seq):
    letter_index = list(seq)
    encoded_seq = aa_props2.loc[letter_index].values
    return encoded_seq.astype(np.float32)

In [10]:
def aggre(s):
    if type(s.values[0]) == str:
        return s.values[0]
    return np.mean(s)

In [11]:
df = pd.read_csv('/kaggle/input/train-v2-csv/train_v2.csv')
df

,mutant_seq,Normalized Activity,Normalized Selectivity
0,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.000,1.000
1,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,3.228,1.837
2,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,2.170,2.445
3,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.759,1.061
4,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.531,1.032
...,...,...,...
1589,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.044,1.879
1590,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.042,2.051
1591,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.045,1.939
1592,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.038,1.887


In [12]:
root_dir = 'train_dataset'
os.makedirs(root_dir, exist_ok=True)
pdb_seq = "MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFARTCKCLAEHFNVVLFDLPFAGQSRQHNPQRGLITKDDEVEILLALIERFEVNHLVSASWGGISTLLALSRNPRGIRSSVVMAFAPGLNQAMLDYVGRAQALIELDDKSAIGHLLNETVGKYLPQRLKASNHQHMASLATGEYEQARFHIDQVLALNDRGYLACLERIQSHVHFINGSWDEYTTAEDARQFRDYLPHCSFSRVEGTGHFLDLESKLAAVRVHRALLEHLLKQPEPQRAERAAGFHEMAIGYA"

df['mutation_feature_path'] = None
df['wildtype_feature_path'] = None
df['graph_index'] = None

graphs = []

pdb = None

for index,row in df.iterrows(): 
    graph = generate_graph("/kaggle/input/af-q51559-f1-model-v4-pdb/AF-Q51559-F1-model_v4.pdb")
    graphs.append(graph)
    graph_index = len(graphs) - 1

    encoded_pdb_seq = seq_encode(pdb_seq).cpu().numpy()
    wildtype_feature_path = f'wildtype_feature_{graph_index}'
    phys_chem_enc_seq = phys_chem_encode(pdb_seq)

    encoded_pdb_seq = np.concatenate([encoded_pdb_seq , phys_chem_enc_seq], axis=-1)
    wildtype_feature_path = 'wildtype_feature'
    np.savez_compressed(root_dir+'/'+wildtype_feature_path, wildtype_seq=encoded_pdb_seq)
    
    #后面这一串代码做的事实际上就是获取突变序列，我实际上可以再做一个单独的数据处理cell便于直接获得
    pdb_mut_seq = row["mutant_seq"]
    encoded_pdb_mut_seq = seq_encode(pdb_mut_seq).cpu().numpy()
    phys_chem_enc_seq = phys_chem_encode(pdb_mut_seq)
    encoded_pdb_mut_seq = np.concatenate([encoded_pdb_mut_seq, phys_chem_enc_seq], axis=-1)
    mutation_pos = np.zeros(graph.num_nodes())
    position = [74, 101, 143, 148, 173, 176]
    for pdb_position_index in position:
        mutation_pos[pdb_position_index-1] = 1
    
    
    mutation_feature_path = f'mutation_feature_{index}'
    np.savez_compressed(root_dir+'/'+mutation_feature_path, mutation_seq=encoded_pdb_mut_seq, mutation_pos=mutation_pos)
    
    df.loc[index, 'graph_index'] = graph_index
    df.loc[index, 'mutation_feature_path'] = mutation_feature_path+'.npz'
    df.loc[index, 'wildtype_feature_path'] = wildtype_feature_path+'.npz'


df.to_csv(root_dir+'/overview_df.csv')

dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)


In [13]:
df

,mutant_seq,Normalized Activity,Normalized Selectivity,mutation_feature_path,wildtype_feature_path,graph_index
0,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.000,1.000,mutation_feature_0.npz,wildtype_feature.npz,0
1,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,3.228,1.837,mutation_feature_1.npz,wildtype_feature.npz,1
2,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,2.170,2.445,mutation_feature_2.npz,wildtype_feature.npz,2
3,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.759,1.061,mutation_feature_3.npz,wildtype_feature.npz,3
4,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,1.531,1.032,mutation_feature_4.npz,wildtype_feature.npz,4
...,...,...,...,...,...,...
1589,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.044,1.879,mutation_feature_1589.npz,wildtype_feature.npz,1589
1590,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.042,2.051,mutation_feature_1590.npz,wildtype_feature.npz,1590
1591,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.045,1.939,mutation_feature_1591.npz,wildtype_feature.npz,1591
1592,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...,0.038,1.887,mutation_feature_1592.npz,wildtype_feature.npz,1592


# Test Data

In [14]:
df = pd.read_csv('/kaggle/input/test-v1-csv/test_v1.csv')

In [15]:
df

,mutant_seq
0,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
1,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
2,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
3,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
4,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
...,...
920,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
921,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
922,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...
923,MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFART...


In [16]:
root_dir = 'test_dataset'
os.makedirs(root_dir, exist_ok=True)
pdb_seq = "MRRESLLVSVCKGLRVHVERVGQDPGRSTVMLVNGAMATTASFARTCKCLAEHFNVVLFDLPFAGQSRQHNPQRGLITKDDEVEILLALIERFEVNHLVSASWGGISTLLALSRNPRGIRSSVVMAFAPGLNQAMLDYVGRAQALIELDDKSAIGHLLNETVGKYLPQRLKASNHQHMASLATGEYEQARFHIDQVLALNDRGYLACLERIQSHVHFINGSWDEYTTAEDARQFRDYLPHCSFSRVEGTGHFLDLESKLAAVRVHRALLEHLLKQPEPQRAERAAGFHEMAIGYA"

df['mutation_feature_path'] = None
df['wildtype_feature_path'] = None
df['graph_index'] = None

graphs = []

pdb = None

for index,row in df.iterrows(): 
    graph = generate_graph("/kaggle/input/af-q51559-f1-model-v4-pdb/AF-Q51559-F1-model_v4.pdb")
    graphs.append(graph)
    graph_index = len(graphs) - 1

    encoded_pdb_seq = seq_encode(pdb_seq).cpu().numpy()
    wildtype_feature_path = f'wildtype_feature_{graph_index}'
    phys_chem_enc_seq = phys_chem_encode(pdb_seq)

    encoded_pdb_seq = np.concatenate([encoded_pdb_seq , phys_chem_enc_seq], axis=-1)
    wildtype_feature_path = 'wildtype_feature'
    np.savez_compressed(root_dir+'/'+wildtype_feature_path, wildtype_seq=encoded_pdb_seq)
    
    #后面这一串代码做的事实际上就是获取突变序列，我实际上可以再做一个单独的数据处理cell便于直接获得
    pdb_mut_seq = row["mutant_seq"]
    encoded_pdb_mut_seq = seq_encode(pdb_mut_seq).cpu().numpy()
    phys_chem_enc_seq = phys_chem_encode(pdb_mut_seq)
    encoded_pdb_mut_seq = np.concatenate([encoded_pdb_mut_seq, phys_chem_enc_seq], axis=-1)
    mutation_pos = np.zeros(graph.num_nodes())
    position = [74, 101, 143, 148, 173, 176]
    for pdb_position_index in position:
        mutation_pos[pdb_position_index - 1] = 1
    
    
    mutation_feature_path = f'mutation_feature_{index}'
    np.savez_compressed(root_dir+'/'+mutation_feature_path, mutation_seq=encoded_pdb_mut_seq, mutation_pos=mutation_pos)
    
    df.loc[index, 'graph_index'] = graph_index
    df.loc[index, 'mutation_feature_path'] = mutation_feature_path+'.npz'
    df.loc[index, 'wildtype_feature_path'] = wildtype_feature_path+'.npz'


df.to_csv(root_dir+'/overview_df.csv')

dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)
